# DDRNet+hardnet+DetailGT

Output:
1. `seghead(FFM(DAPPPM, layer5_)` - main
2. `seghead(layer3_ + upsample(layer3))`
3. `seghead(layer4_ + upsample(layer4))`

In [ ]:
import sys
sys.path.insert(0, '.')
import os
import logging
import random
import time
import math
import torch
import numpy as np 
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import init
from torch.utils import data
from datetime import datetime, timedelta
from collections import OrderedDict

from torchsummary import summary

import torch.distributed as dist

 
!pip install tensorboardX
from tensorboardX import SummaryWriter

     |████████████████████████████████| 124 kB 12.5 MB/s 


In [ ]:
BatchNorm2d = nn.BatchNorm2d
ReLU = nn.ReLU
AvgPool2d = nn.AvgPool2d
AdaptiveAvgPool2d = nn.AdaptiveAvgPool2d

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
batch_size = 8
n_workers = 2
print_interval=10
val_interval=500

n_classes = 19

model_arch = 'ddrnet_slim_23'

bn_mom = 0.1

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
float_tensor_type = torch.cuda.FloatTensor if device.type=='cuda' else torch.FloatTensor

---
## Network

In [ ]:
class ConvBNReLU(nn.Module):
    def __init__(self, in_chan, out_chan, ks=3, stride=1, padding=1, need_relu=True, *args, **kwargs):
        super(ConvBNReLU, self).__init__()
        self.conv = nn.Conv2d(in_chan,
                out_chan,
                kernel_size = ks,
                stride = stride,
                padding = padding,
                bias = False)
        # self.bn = BatchNorm2d(out_chan)
        self.bn = BatchNorm2d(out_chan)
        self.need_relu = need_relu
        if need_relu:
            self.relu = nn.ReLU(inplace=True)
        self.init_weight()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.need_relu:
            x = self.relu(x)
        return x

    def init_weight(self):
        for ly in self.children():
            if isinstance(ly, nn.Conv2d):
                nn.init.kaiming_normal_(ly.weight, a=1)
                if not ly.bias is None: nn.init.constant_(ly.bias, 0)

    def get_params(self):
        wd_params, nowd_params = [], []
        for name, module in self.named_modules():
            if isinstance(module, nn.Conv2d):
                wd_params.append(module.weight)
                if not module.bias is None:
                    nowd_params.append(module.bias)
            elif isinstance(module, (BatchNorm2d, ReLU)):
                nowd_params += list(module.parameters())
        return wd_params, nowd_params

In [ ]:
class StepDAPPM(nn.Module):
    def __init__(self, pool_ks, pool_stride, pool_padding, bn_inplanes, conv_out_planes, conv_ks=1):
        super(StepDAPPM, self).__init__()
        self.pool = None
        if pool_ks > 1:
            self.pool = nn.AvgPool2d(kernel_size=pool_ks, stride=pool_stride, padding=pool_padding)
        elif pool_ks == 1:
            self.pool = nn.AdaptiveAvgPool2d((1, 1))

        self.bn = BatchNorm2d(bn_inplanes, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(bn_inplanes, conv_out_planes, kernel_size=conv_ks, padding=conv_ks//2, bias=False)

    def forward(self, x):
        if self.pool is not None:
            x = self.pool(x)
        x = self.conv(self.relu(self.bn(x)))
        return x
    
    def get_params(self):
        wd_params, nowd_params = [], []
        for name, module in self.named_modules():
            if isinstance(module, nn.Conv2d):
                wd_params.append(module.weight)
                if not module.bias is None:
                    nowd_params.append(module.bias)
            elif isinstance(module, (BatchNorm2d, ReLU)):
                nowd_params += list(module.parameters())
        return wd_params, nowd_params

In [ ]:
class DAPPM(nn.Module):
    def __init__(self, inplanes, branch_planes, outplanes):
        super(DAPPM, self).__init__()
        self.scale1 = StepDAPPM( 5, 2, 2, inplanes, branch_planes, conv_ks=1)
        self.scale2 = StepDAPPM( 9, 4, 4, inplanes, branch_planes, conv_ks=1)
        self.scale3 = StepDAPPM(17, 8, 8, inplanes, branch_planes, conv_ks=1)
        self.scale4 = StepDAPPM( 1, 0, 0, inplanes, branch_planes, conv_ks=1)
        self.scale0 = StepDAPPM( 0, 0, 0, inplanes, branch_planes, conv_ks=1)

        self.process1 = StepDAPPM(0, 0, 0, branch_planes, branch_planes, conv_ks=3)
        self.process2 = StepDAPPM(0, 0, 0, branch_planes, branch_planes, conv_ks=3)
        self.process3 = StepDAPPM(0, 0, 0, branch_planes, branch_planes, conv_ks=3)
        self.process4 = StepDAPPM(0, 0, 0, branch_planes, branch_planes, conv_ks=3)
        
        self.compression = StepDAPPM(0, 0, 0, branch_planes * 5, outplanes, conv_ks=1)
        self.shortcut = StepDAPPM(0, 0, 0, inplanes, outplanes, conv_ks=1)
        
    def forward(self, x):

        #x = self.downsample(x)
        width = x.shape[-1]
        height = x.shape[-2]
        x_list = []

        x_list.append(self.scale0(x))
        x_list.append(self.process1((F.interpolate(self.scale1(x),
                        size=[height, width],
                        mode='bilinear', align_corners=True)+x_list[0])))
        x_list.append((self.process2((F.interpolate(self.scale2(x),
                        size=[height, width],
                        mode='bilinear', align_corners=True)+x_list[1]))))
        x_list.append(self.process3((F.interpolate(self.scale3(x),
                        size=[height, width],
                        mode='bilinear', align_corners=True)+x_list[2])))
        x_list.append(self.process4((F.interpolate(self.scale4(x),
                        size=[height, width],
                        mode='bilinear', align_corners=True)+x_list[3])))
       
        out = self.compression(torch.cat(x_list, 1)) + self.shortcut(x)
        return out    

    def get_params(self):
        wd_params, nowd_params = [], []
        for name, child in self.named_children():
            child_wd_params, child_nowd_params = child.get_params() \
                if not isinstance(child, ReLU) else ([],[])
            wd_params += child_wd_params
            nowd_params += child_nowd_params
        return wd_params, nowd_params

In [ ]:
class segmenthead(nn.Module):

    def __init__(self, inplanes, interplanes, outplanes, scale_factor=None):
        super(segmenthead, self).__init__()
        self.bn1 = BatchNorm2d(inplanes, momentum=bn_mom)
        self.conv1 = nn.Conv2d(inplanes, interplanes, kernel_size=3, padding=1, bias=False)
        self.bn2 = BatchNorm2d(interplanes, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(interplanes, outplanes, kernel_size=1, padding=0, bias=True)
        self.scale_factor = scale_factor

    def forward(self, x):
        
        x = self.conv1(self.relu(self.bn1(x)))
        out = self.conv2(self.relu(self.bn2(x)))

        if self.scale_factor is not None:
            height = x.shape[-2] * self.scale_factor
            width = x.shape[-1] * self.scale_factor
            out = F.interpolate(out,
                        size=[height, width],
                        mode='bilinear', align_corners=True)

        return out
    
    def get_params(self):
        wd_params, nowd_params = [], []
        for name, module in self.named_modules():
            if isinstance(module, (nn.Conv2d)):
                wd_params.append(module.weight)
                if not module.bias is None:
                    nowd_params.append(module.bias)
            elif isinstance(module, (BatchNorm2d, ReLU)):
                nowd_params += list(module.parameters())
        return wd_params, nowd_params

In [ ]:
class FeatureFusionModule(nn.Module):
    def __init__(self, in_chan, out_chan, *args, **kwargs):
        super(FeatureFusionModule, self).__init__()
        self.convblk = ConvBNReLU(in_chan, out_chan, ks=1, stride=1, padding=0)
        self.conv1 = nn.Conv2d(out_chan,
                out_chan//4,
                kernel_size = 1,
                stride = 1,
                padding = 0,
                bias = False)
        self.conv2 = nn.Conv2d(out_chan//4,
                out_chan,
                kernel_size = 1,
                stride = 1,
                padding = 0,
                bias = False)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
        self.init_weight()

    def forward(self, fsp, fcp):
        fcat = torch.cat([fsp, fcp], dim=1)
        feat = self.convblk(fcat)
        atten = F.avg_pool2d(feat, feat.size()[2:])
        atten = self.conv1(atten)
        atten = self.relu(atten)
        atten = self.conv2(atten)
        atten = self.sigmoid(atten)
        feat_atten = torch.mul(feat, atten)
        feat_out = feat_atten + feat
        return feat_out

    def init_weight(self):
        for ly in self.children():
            if isinstance(ly, nn.Conv2d):
                nn.init.kaiming_normal_(ly.weight, a=1)
                if not ly.bias is None: nn.init.constant_(ly.bias, 0)

    def get_params(self):
        wd_params, nowd_params = [], []
        for name, module in self.named_modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                wd_params.append(module.weight)
                if not module.bias is None:
                    nowd_params.append(module.bias)
            elif isinstance(module, (BatchNorm2d, ReLU)):
                nowd_params += list(module.parameters())
        return wd_params, nowd_params

In [ ]:
class HarDBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, grmul, n_layers, keepBase=False):
        super(HarDBlock, self).__init__()
        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.grmul = grmul
        self.links = []
        self.out_channels = 0
        self.keepBase = keepBase
        self.layers = nn.ModuleList([])
        for i in range(n_layers):
            out_ch, in_ch, el_links = self.get_links(i+1)
            self.links.append(el_links)
            self.layers.append(ConvBNReLU(in_ch, out_ch, need_relu=(n_layers - 1 != i)))
            if (i % 2 == 0) or (i == n_layers - 1):
                self.out_channels += out_ch

    def __out_ch(self, layer_id): 
        out_ch = self.growth_rate
        for i in range(1, int(math.log2(layer_id))+1):
            if layer_id % 2**i == 0:
                out_ch = out_ch*self.grmul
        return int(int(out_ch + 1) / 2) * 2

    def get_links(self, layer_id):
        in_ch = 0
        links_ = []
        for i in range(int(math.log2(layer_id))):
            diff = 2**i
            if (layer_id % diff == 0) and layer_id - diff > 0:
                in_ch += self.__out_ch(layer_id - diff)
                links_.append(layer_id - diff)
        if math.log2(layer_id).is_integer():
            in_ch += self.in_channels
            links_.append(0)
        return self.__out_ch(layer_id), in_ch, links_

    def forward(self, x):
        data = [x]
        for layer in range(len(self.layers)):
            layer_input = []
            for link in self.links[layer]:
                layer_input.append(data[link])
            in_ = layer_input[0] if len(layer_input) == 1 \
                else torch.cat(layer_input, dim=1)
            data.append(self.layers[layer](in_))
        t = len(data)
        out = []
        for i in range(t):
            if (i % 2 == 1) or (self.keepBase and i == 0) or (i == t-1):
                out.append(data[i])
        return torch.cat(out, dim=1)
    
    def get_out_ch(self):
        return self.out_channels

    def get_params(self):
        wd_params, nowd_params = [], []
        for name, child in self.layers.named_children():
            child_wd_params, child_nowd_params = child.get_params() \
                if not isinstance(child, ReLU) else ([],[])
            wd_params += child_wd_params
            nowd_params += child_nowd_params
        return wd_params, nowd_params

In [ ]:
class ConvHarDBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1, growth_rate=10, grmul=1.7, n_layers=4):
        super(ConvHarDBlock, self).__init__()
        self.layers = nn.ModuleList([])
        if stride != 1:
            self.layers.append(ConvBNReLU(inplanes, planes, ks=1, stride=stride,\
                                          padding=0, need_relu=True))
        hardblock_inplanes = planes if stride != 1 else inplanes
        hardblock = HarDBlock(hardblock_inplanes, growth_rate, grmul, n_layers)
        self.layers.append(hardblock)

        self.layers.append(ConvBNReLU(hardblock.get_out_ch(), planes, ks=1, \
                                      stride=1, padding=0, need_relu=True))
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x        

    def get_params(self):
        wd_params, nowd_params = [], []
        for name, child in self.layers.named_children():
            child_wd_params, child_nowd_params = child.get_params() \
                if not isinstance(child, ReLU) else ([],[])
            wd_params += child_wd_params
            nowd_params += child_nowd_params
        return wd_params, nowd_params

In [ ]:
class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None, no_relu=True):
        super(Bottleneck, self).__init__()
        self.blk1 = ConvBNReLU(inplanes, planes, ks=1, stride=1, padding=0)
        self.blk2 = ConvBNReLU(planes, planes, ks=3, stride=stride, padding=1)
        self.blk3 = ConvBNReLU(planes, planes * self.expansion, ks=1, stride=1, padding=0, need_relu=False)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.no_relu = no_relu

    def forward(self, x):
        # print('input', x.shape)
        residual = x

        out = self.blk1(x)
        out = self.blk2(out)
        out = self.blk3(out)
        # print('out', out.shape)
        
        if self.downsample is not None:
            # print('down', self.downsample)
            residual = self.downsample(x)
        # print('res', residual.shape)
        out += residual
        if self.no_relu:
            return out
        else:
            return self.relu(out)
    
    def get_params(self):
        wd_params, nowd_params = [], []
        for name, child in self.named_children():
            child_wd_params, child_nowd_params = child.get_params() \
                if not isinstance(child, ReLU) else ([],[])
            wd_params += child_wd_params
            nowd_params += child_nowd_params
        return wd_params, nowd_params

In [ ]:
class DualResNet(nn.Module):

    def __init__(self, num_classes=19, planes=64, spp_planes=128, head_planes=128, augment=False):
        super(DualResNet, self).__init__()

        growth_rate = [  10,16,18,24,32]
        highres_planes = planes * 2
        self.augment = augment


        self.conv0 = ConvBNReLU(     3, planes, ks=3, stride=2, padding=1)
        self.conv1 = ConvBNReLU(planes, planes, ks=3, stride=2, padding=1)

        self.relu = nn.ReLU(inplace=False)
        
        self.layer1 = ConvHarDBlock(planes,     planes,     stride=1, growth_rate=growth_rate[0])
        self.layer2 = ConvHarDBlock(planes,     planes * 2, stride=2, growth_rate=growth_rate[1])
        self.layer3 = ConvHarDBlock(planes * 2, planes * 4, stride=2, growth_rate=growth_rate[2])
        self.layer4 = ConvHarDBlock(planes * 4, planes * 8, stride=2, growth_rate=growth_rate[3])


        self.compression3 = ConvBNReLU(planes * 4, highres_planes, ks=1, stride=1, padding=0, need_relu=False)
        self.compression4 = ConvBNReLU(planes * 8, highres_planes, ks=1, stride=1, padding=0, need_relu=False)

        self.down3 = ConvBNReLU(highres_planes, planes * 4, ks=3, stride=2, padding=1, need_relu=False)
        self.down4_0 = ConvBNReLU(highres_planes, planes * 4, ks=3, stride=2, padding=1, need_relu=True)
        self.down4_1 = ConvBNReLU(planes * 4, planes * 8, ks=3, stride=2, padding=1, need_relu=False)

        self.layer3_ = ConvHarDBlock(planes * 2, highres_planes, stride=1, growth_rate=growth_rate[2])
        
        self.layer4_ = ConvHarDBlock(highres_planes, highres_planes, stride=1, growth_rate=growth_rate[2])

        self.layer5_ = self._make_bottleneck(highres_planes, highres_planes)

        self.layer5 =  self._make_bottleneck(planes * 8, planes * 8, stride=2)

        self.spp = DAPPM(planes * 8 * Bottleneck.expansion, spp_planes, planes * 4)

        self.ffm = FeatureFusionModule(planes * 4 + highres_planes * 2, planes * 4)

        self.seghead_out1 = segmenthead(planes * 4, head_planes, num_classes)
        self.seghead_out2 = segmenthead(highres_planes, head_planes, 1)
        self.seghead_out3 = segmenthead(highres_planes, head_planes, num_classes)            

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_bottleneck(self, inplanes, planes, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * Bottleneck.expansion:
            downsample = ConvBNReLU(inplanes, planes * Bottleneck.expansion, ks=1, stride=stride, padding=0, need_relu=False)
            # print(stride, inplanes, planes * Bottleneck.expansion, downsample)
        return Bottleneck(inplanes, planes, stride, downsample)


    def forward(self, x):

        width, height = x.shape[-1], x.shape[-2]
        width_output, height_output = x.shape[-1] // 8, x.shape[-2] // 8

        layers = []

        x = self.conv0(x)
        x = self.conv1(x)

        x = self.layer1(x)
        layers.append(x)

        x = self.layer2(self.relu(x))
        layers.append(x)
  
        x = self.layer3(self.relu(x))
        layers.append(x)
        x_ = self.layer3_(self.relu(layers[1]))

        x = x + self.down3(self.relu(x_))
        step_res = self.compression3(self.relu(layers[2]))
        x_ = x_ + F.interpolate(
                        step_res,
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=True)
        out_2 = x_

        x = self.layer4(self.relu(x))
        layers.append(x)
        
        x_ = self.layer4_(self.relu(x_))

        x = x + self.down4_1(self.down4_0(self.relu(x_)))
        x_ = x_ + F.interpolate(
                        self.compression4(self.relu(layers[3])),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=True)
        
        out_3 = x_

        x_ = self.layer5_(self.relu(x_))
        x = F.interpolate(
                        self.spp(self.layer5(self.relu(x))),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=True)
        
        out_1 = self.ffm(x, x_)
        
        out_1 = self.seghead_out1(out_1)
        out_2 = self.seghead_out2(out_2)
        out_3 = self.seghead_out3(out_3)

        out_1 = F.interpolate(out_1, (height, width), mode='bilinear', align_corners=True)
        out_2 = F.interpolate(out_2, (height, width), mode='bilinear', align_corners=True)
        out_3 = F.interpolate(out_3, (height, width), mode='bilinear', align_corners=True)
        
        return out_1, out_2, out_3


    def get_params(self):
        wd_params, nowd_params = [], []
        for name, child in self.named_children():
            child_wd_params, child_nowd_params = child.get_params() \
                if not isinstance(child, ReLU) else ([],[])
            wd_params += child_wd_params
            nowd_params += child_nowd_params
        return wd_params, nowd_params

In [ ]:
# net = DualResNet(num_classes=19, planes=64, spp_planes=128, head_planes=128, augment=False)

In [ ]:
# net.get_params()

In [ ]:
# torch.save(net.state_dict(), 'model_maxmIOU103.pth')

---
## Loss & Optimizer

In [ ]:
def dice_loss_func(input, target):
    smooth = 1.
    n = input.size(0)
    iflat = input.view(n, -1)
    tflat = target.view(n, -1)
    intersection = (iflat * tflat).sum(1)
    loss = 1 - ((2. * intersection + smooth) /
                (iflat.sum(1) + tflat.sum(1) + smooth))
    return loss.mean()


In [ ]:
class OhemCELoss(nn.Module):
    def __init__(self, thresh, n_min, ignore_lb=255, *args, **kwargs):
        super(OhemCELoss, self).__init__()
        self.thresh = -torch.log(torch.tensor(thresh, dtype=torch.float)).to(device)
        self.n_min = n_min
        self.ignore_lb = ignore_lb
        self.criteria = nn.CrossEntropyLoss(ignore_index=ignore_lb, reduction='none')

    def forward(self, logits, labels):
        N, C, H, W = logits.size()
        loss = self.criteria(logits, labels).view(-1)
        loss, _ = torch.sort(loss, descending=True)
        if loss[self.n_min] > self.thresh:
            loss = loss[loss>self.thresh]
        else:
            loss = loss[:self.n_min]
        return torch.mean(loss)

In [ ]:
class DetailAggregateLoss(nn.Module):
    def __init__(self, *args, **kwargs):
        super(DetailAggregateLoss, self).__init__()
        
        self.laplacian_kernel = torch.tensor(
            [-1, -1, -1, -1, 8, -1, -1, -1, -1],
            dtype=torch.float32).reshape(1, 1, 3, 3).requires_grad_(False).type(float_tensor_type)        

        self.fuse_kernel = torch.nn.Parameter(torch.tensor([[6./10], [3./10], [1./10]],
            dtype=torch.float32).reshape(1, 3, 1, 1).type(float_tensor_type))

    def forward(self, boundary_logits, gtmasks):

        boundary_targets = F.conv2d(gtmasks.unsqueeze(1).type(float_tensor_type), self.laplacian_kernel, padding=1)
        boundary_targets = boundary_targets.clamp(min=0)
        boundary_targets[boundary_targets > 0.1] = 1
        boundary_targets[boundary_targets <= 0.1] = 0

        boundary_targets_x2 = F.conv2d(gtmasks.unsqueeze(1).type(float_tensor_type), self.laplacian_kernel, stride=2, padding=1)
        boundary_targets_x2 = boundary_targets_x2.clamp(min=0)
        
        boundary_targets_x4 = F.conv2d(gtmasks.unsqueeze(1).type(float_tensor_type), self.laplacian_kernel, stride=4, padding=1)
        boundary_targets_x4 = boundary_targets_x4.clamp(min=0)

        boundary_targets_x4_up = F.interpolate(boundary_targets_x4, boundary_targets.shape[2:], mode='nearest')
        boundary_targets_x2_up = F.interpolate(boundary_targets_x2, boundary_targets.shape[2:], mode='nearest')
        
        boundary_targets_x2_up[boundary_targets_x2_up > 0.1] = 1
        boundary_targets_x2_up[boundary_targets_x2_up <= 0.1] = 0
        
        
        boundary_targets_x4_up[boundary_targets_x4_up > 0.1] = 1
        boundary_targets_x4_up[boundary_targets_x4_up <= 0.1] = 0
       
        boudary_targets_pyramids = torch.stack((boundary_targets, boundary_targets_x2_up, boundary_targets_x4_up), dim=1)
        
        boudary_targets_pyramids = boudary_targets_pyramids.squeeze(2)
        boudary_targets_pyramid = F.conv2d(boudary_targets_pyramids, self.fuse_kernel)

        boudary_targets_pyramid[boudary_targets_pyramid > 0.1] = 1
        boudary_targets_pyramid[boudary_targets_pyramid <= 0.1] = 0
        
        
        if boundary_logits.shape[-1] != boundary_targets.shape[-1]:
            boundary_logits = F.interpolate(
                boundary_logits, boundary_targets.shape[2:], mode='bilinear', align_corners=True)
        
        bce_loss = F.binary_cross_entropy_with_logits(boundary_logits, boudary_targets_pyramid)
        dice_loss = dice_loss_func(torch.sigmoid(boundary_logits), boudary_targets_pyramid)
        return bce_loss,  dice_loss

    def get_params(self):
        wd_params, nowd_params = [], []
        for name, module in self.named_modules():
                nowd_params += list(module.parameters())
        return nowd_params

In [ ]:
class Optimizer(object):
    def __init__(self, model, loss, lr0, momentum, wd, warmup_steps, 
                 warmup_start_lr, max_iter, power, *args, **kwargs):
        self.warmup_steps = warmup_steps
        self.warmup_start_lr = warmup_start_lr
        self.lr0 = lr0
        self.lr = self.lr0
        self.max_iter = float(max_iter)
        self.power = power
        self.it = 0
        wd_params, nowd_params = model.get_params()
        loss_nowd_params = loss.get_params()
        param_list = [
                {'params': wd_params},
                {'params': nowd_params, 'weight_decay': 0},
                {'params': loss_nowd_params}]
        self.optim = torch.optim.SGD(
                param_list,
                # model.parameters(),
                lr = lr0,
                momentum = momentum,
                weight_decay = wd)
        self.warmup_factor = (self.lr0/self.warmup_start_lr)**(1./self.warmup_steps)

    def get_lr(self):
        if self.it <= self.warmup_steps:
            lr = self.warmup_start_lr*(self.warmup_factor**self.it)
        else:
            factor = (1-(self.it-self.warmup_steps)/(self.max_iter-self.warmup_steps))**self.power
            lr = self.lr0 * factor
        return lr

    def step(self):
        self.lr = self.get_lr()
        for pg in self.optim.param_groups:
            if pg.get('lr_mul', False):
                pg['lr'] = self.lr * 10
            else:
                pg['lr'] = self.lr
        if self.optim.defaults.get('lr_mul', False):
            self.optim.defaults['lr'] = self.lr * 10
        else:
            self.optim.defaults['lr'] = self.lr
        self.it += 1
        self.optim.step()
        if self.it == self.warmup_steps+2:
            logger.info('==> warmup done, start to implement poly lr strategy')
    
    def get_state(self):
        return {
            'warmup_steps': self.warmup_steps,
            'warmup_start_lr': self.warmup_start_lr,
            'lr0': self.lr0,
            'lr': self.lr,
            'max_iter': self.max_iter,
            'power': self.power, 
            'it': self.it,
            'optim_state': self.optim.state_dict(),
            'warmup_factor': self.warmup_factor
        }

    def load_state(self, state):
        self.warmup_steps = state.get('warmup_steps')
        self.warmup_start_lr = state.get('warmup_start_lr')
        self.lr0 = state.get('lr0')
        self.lr = state.get('lr')
        self.max_iter = state.get('max_iter')
        self.power = state.get('power')
        self.it = state.get('it')
        self.optim.load_state_dict(state.get('optim_state'))
        self.warmup_factor = state.get('warmup_factor')


    def zero_grad(self):
        self.optim.zero_grad()

In [ ]:
score_thres = 0.7
n_img_per_gpu = 8
cropsize = (512, 1024)
n_min = n_img_per_gpu*cropsize[0]*cropsize[1]//32
ignore_idx=255

---
## Dataset

In [ ]:
!wget https://raw.githubusercontent.com/MichaelFan01/STDC-Seg/master/cityscapes_info.json

--2021-11-26 13:39:11--  https://raw.githubusercontent.com/MichaelFan01/STDC-Seg/master/cityscapes_info.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7412 (7.2K) [text/plain]
Saving to: ‘cityscapes_info.json’

cityscapes_info.jso 100%[===================>]   7.24K  --.-KB/s    in 0s      

2021-11-26 13:39:11 (55.9 MB/s) - ‘cityscapes_info.json’ saved [7412/7412]



In [ ]:
from PIL import Image
import PIL.ImageEnhance as ImageEnhance
import random
import numpy as np


class RandomCrop(object):
    def __init__(self, size, *args, **kwargs):
        self.size = size

    def __call__(self, im_lb):
        im = im_lb['im']
        lb = im_lb['lb']
        assert im.size == lb.size
        W, H = self.size
        w, h = im.size

        if (W, H) == (w, h): return dict(im=im, lb=lb)
        if w < W or h < H:
            scale = float(W) / w if w < h else float(H) / h
            w, h = int(scale * w + 1), int(scale * h + 1)
            im = im.resize((w, h), Image.BILINEAR)
            lb = lb.resize((w, h), Image.NEAREST)
        sw, sh = random.random() * (w - W), random.random() * (h - H)
        crop = int(sw), int(sh), int(sw) + W, int(sh) + H
        return dict(
                im = im.crop(crop),
                lb = lb.crop(crop)
                    )


class HorizontalFlip(object):
    def __init__(self, p=0.5, *args, **kwargs):
        self.p = p

    def __call__(self, im_lb):
        if random.random() > self.p:
            return im_lb
        else:
            im = im_lb['im']
            lb = im_lb['lb']
            return dict(im = im.transpose(Image.FLIP_LEFT_RIGHT),
                        lb = lb.transpose(Image.FLIP_LEFT_RIGHT),
                    )


class RandomScale(object):
    def __init__(self, scales=(1, ), *args, **kwargs):
        self.scales = scales
        # print('scales: ', scales)

    def __call__(self, im_lb):
        im = im_lb['im']
        lb = im_lb['lb']
        W, H = im.size
        scale = random.choice(self.scales)
        # scale = np.random.uniform(min(self.scales), max(self.scales))
        w, h = int(W * scale), int(H * scale)
        return dict(im = im.resize((w, h), Image.BILINEAR),
                    lb = lb.resize((w, h), Image.NEAREST),
                )


class ColorJitter(object):
    def __init__(self, brightness=None, contrast=None, saturation=None, *args, **kwargs):
        if not brightness is None and brightness>0:
            self.brightness = [max(1-brightness, 0), 1+brightness]
        if not contrast is None and contrast>0:
            self.contrast = [max(1-contrast, 0), 1+contrast]
        if not saturation is None and saturation>0:
            self.saturation = [max(1-saturation, 0), 1+saturation]

    def __call__(self, im_lb):
        im = im_lb['im']
        lb = im_lb['lb']
        r_brightness = random.uniform(self.brightness[0], self.brightness[1])
        r_contrast = random.uniform(self.contrast[0], self.contrast[1])
        r_saturation = random.uniform(self.saturation[0], self.saturation[1])
        im = ImageEnhance.Brightness(im).enhance(r_brightness)
        im = ImageEnhance.Contrast(im).enhance(r_contrast)
        im = ImageEnhance.Color(im).enhance(r_saturation)
        return dict(im = im,
                    lb = lb,
                )


class MultiScale(object):
    def __init__(self, scales):
        self.scales = scales

    def __call__(self, img):
        W, H = img.size
        sizes = [(int(W*ratio), int(H*ratio)) for ratio in self.scales]
        imgs = []
        [imgs.append(img.resize(size, Image.BILINEAR)) for size in sizes]
        return imgs


class Compose(object):
    def __init__(self, do_list):
        self.do_list = do_list

    def __call__(self, im_lb):
        for comp in self.do_list:
            im_lb = comp(im_lb)
        return im_lb

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import os.path as osp
import os
from PIL import Image
import numpy as np
import json


class CityScapes(Dataset):
    def __init__(self, rootpth, cropsize=(640, 480), mode='train', 
    randomscale=(0.125, 0.25, 0.375, 0.5, 0.675, 0.75, 0.875, 1.0, 1.25, 1.5), *args, **kwargs):
        super(CityScapes, self).__init__(*args, **kwargs)
        assert mode in ('train', 'val', 'test', 'trainval')
        self.mode = mode
        print('self.mode', self.mode)
        self.ignore_lb = 255

        with open('./cityscapes_info.json', 'r') as fr:
            labels_info = json.load(fr)
        self.lb_map = {el['id']: el['trainId'] for el in labels_info}
        

        ## parse img directory
        self.imgs = {}
        imgnames = []
        impth = osp.join(rootpth, 'leftImg8bit', mode)
        folders = os.listdir(impth)
        for fd in folders:
            fdpth = osp.join(impth, fd)
            im_names = os.listdir(fdpth)
            names = [el.replace('_leftImg8bit.png', '') for el in im_names]
            impths = [osp.join(fdpth, el) for el in im_names]
            imgnames.extend(names)
            self.imgs.update(dict(zip(names, impths)))

        ## parse gt directory
        self.labels = {}
        gtnames = []
        gtpth = osp.join(rootpth, 'gtFine', mode)
        folders = os.listdir(gtpth)
        for fd in folders:
            fdpth = osp.join(gtpth, fd)
            lbnames = os.listdir(fdpth)
            lbnames = [el for el in lbnames if 'labelIds' in el]
            names = [el.replace('_gtFine_labelIds.png', '') for el in lbnames]
            lbpths = [osp.join(fdpth, el) for el in lbnames]
            gtnames.extend(names)
            self.labels.update(dict(zip(names, lbpths)))

        self.imnames = imgnames
        self.len = len(self.imnames)
        print('self.len', self.mode, self.len)
        assert set(imgnames) == set(gtnames)
        assert set(self.imnames) == set(self.imgs.keys())
        assert set(self.imnames) == set(self.labels.keys())

        ## pre-processing
        self.to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        self.trans_train = Compose([
            ColorJitter(
                brightness = 0.5,
                contrast = 0.5,
                saturation = 0.5),
            HorizontalFlip(),
            RandomScale(randomscale),
            RandomCrop(cropsize)
            ])


    def __getitem__(self, idx):
        fn  = self.imnames[idx]
        impth = self.imgs[fn]
        lbpth = self.labels[fn]
        img = Image.open(impth).convert('RGB')
        label = Image.open(lbpth)
        if self.mode == 'train' or self.mode == 'trainval':
            im_lb = dict(im = img, lb = label)
            im_lb = self.trans_train(im_lb)
            img, label = im_lb['im'], im_lb['lb']
        img = self.to_tensor(img)
        label = np.array(label).astype(np.int64)[np.newaxis, :]
        label = self.convert_labels(label)
        return img, label


    def __len__(self):
        return self.len


    def convert_labels(self, label):
        for k, v in self.lb_map.items():
            label[label == k] = v
        return label

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dspth = '/content/drive/MyDrive/RnD/datasets/'
cfg_data = {
    'dataset': 'cityscapes',
    'train_split': 'train',
    'val_split': 'val',
    'img_rows': cropsize[0],
    'img_cols': cropsize[1],
    'path': dspth
}
randomscale = (0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5)
ds = CityScapes(cfg_data['path'], cropsize=cropsize, mode='train', randomscale=randomscale)
# sampler = torch.utils.data.distributed.DistributedSampler(ds)
dl = DataLoader(ds,
                batch_size = batch_size,
                shuffle = False,
                # sampler = sampler,
                num_workers = n_workers,
                pin_memory = False,
                drop_last = True)
# exit(0)
dsval = CityScapes(cfg_data['path'], mode='val', randomscale=randomscale)
# sampler_val = torch.utils.data.distributed.DistributedSampler(dsval)
dlval = DataLoader(dsval,
                batch_size = 2,
                shuffle = False,
                # sampler = sampler_val,
                num_workers = n_workers,
                drop_last = False)

## model
ignore_idx = 255

self.mode train
self.len train 2975
self.mode val
self.len val 500


---
# Preparing for train

In [ ]:
best_iou = -100.0
loss_all = 0
loss_n = 0

In [ ]:
def get_logger(logdir):
    logger = logging.getLogger("DDRNet_HBD_B2N_DGT")
    ts = str(datetime.now()).split(".")[0].replace(" ", "_")
    ts = ts.replace(":", "_").replace("-", "_")
    file_path = os.path.join(logdir, "run_{}.log".format(ts))
    hdlr = logging.FileHandler(file_path)
    formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr)
    logger.setLevel(logging.INFO)
    return logger

In [ ]:
!git clone https://github.com/PingoLH/FCHarDNet.git
!cp -r FCHarDNet/ptsemseg ./
!rm -rf FCHarDNet

Cloning into 'FCHarDNet'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 130 (delta 2), reused 7 (delta 1), pack-reused 117
Receiving objects: 100% (130/130), 9.10 MiB | 3.45 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
from ptsemseg.metrics import runningScore, averageMeter

In [ ]:
base_path = "/content/drive/MyDrive/RnD/runs/DDRNet"
model_modification = 'HDB_DGT'
model_modification_path = os.path.join(base_path, model_modification)

In [ ]:
logdir = os.path.join(model_modification_path, str(datetime.fromtimestamp(int(time.time()))))
writer = SummaryWriter(log_dir=logdir)

print("RUNDIR: {}".format(logdir))

logger = get_logger(logdir)

RUNDIR: /content/drive/MyDrive/RnD/runs/DDRNet/HDB_DGT/2021-11-26 13:41:45


In [ ]:
# Setup seeds
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)
np.random.seed(1337)
random.seed(1337)

In [ ]:
running_metrics_val = runningScore(n_classes)
model = DualResNet(num_classes=19, planes=64, 
                         spp_planes=128, head_planes=128, augment=False)
# total_params = sum(p.numel() for p in model.parameters())
# print( 'Parameters:',total_params )
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.apply(weights_init)
pass

In [ ]:
# optimizer init data
momentum = 0.9
weight_decay = 5e-4
lr_start = 1e-2
power = 0.9
warmup_steps = 1000
warmup_start_lr = 1e-5
epoch_iteration = len(ds) // batch_size
max_epoch = 484
# max_epoch = 1
max_iter = max_epoch * epoch_iteration

In [ ]:
start_epoch = 0
it = 0
local_max_epoch = start_epoch + 1 if start_epoch + 1 < max_epoch else max_epoch

In [ ]:
criteria_ffm = OhemCELoss(thresh=score_thres, n_min=n_min, ignore_lb=ignore_idx)    # out1
criteria_layer4 = OhemCELoss(thresh=score_thres, n_min=n_min, ignore_lb=ignore_idx) # out2
boundary_loss_func = DetailAggregateLoss()                                          # out3
criteria_val = OhemCELoss(thresh=score_thres, n_min=n_min, ignore_lb=ignore_idx)    # out1 
val_loss_meter = averageMeter()

In [ ]:
optim = Optimizer(
            model = model.module,
            loss = boundary_loss_func,
            lr0 = lr_start,
            momentum = momentum,
            wd = weight_decay,
            warmup_steps = warmup_steps,
            warmup_start_lr = warmup_start_lr,
            max_iter = max_iter,
            power = power)

In [ ]:
loss_avg = []
loss_boundery_bce = []
loss_boundery_dice = []

---
## Restore state

In [ ]:
runs = sorted(os.listdir(model_modification_path), reverse=True)
best_path = None
last_path = None
for run in runs:
    tmp_base = os.path.join(model_modification_path, run)
    model_name = "{}_{}".format(model_arch, cfg_data['dataset'])
    checkpoint = os.path.join(tmp_base, model_name+'_checkpoint.pkl')
    best = os.path.join(tmp_base, model_name+'_best_model.pkl')
    if not last_path and os.path.exists(checkpoint):
        last_path = checkpoint
    if not best_path and os.path.exists(best):
        best_path = best
    if last_path and best_path:
        break

In [ ]:
best_path, last_path

('/content/drive/MyDrive/RnD/runs/DDRNet/HDB_DGT/2021-11-24 09:22:22/ddrnet_slim_23_cityscapes_best_model.pkl',
 '/content/drive/MyDrive/RnD/runs/DDRNet/HDB_DGT/2021-11-24 09:22:22/ddrnet_slim_23_cityscapes_checkpoint.pkl')

In [ ]:
if best_path and last_path:
    loaded = torch.load(last_path)
    best_iou_arrc = torch.load(best_path)
    model_state = loaded.get('model_state')

    optimizer_state = loaded.get('optimizer_state')
    start_epoch = loaded.get('epoch') + 1

    best_iou = best_iou_arrc.get('best_iou')
    local_max_epoch = start_epoch + 2
    flag = True
    loss_all = 0
    loss_n = 0

    model.load_state_dict(model_state)
    optim.load_state(optimizer_state)

In [ ]:
!tail -n25 '/content/drive/MyDrive/RnD/runs/DDRNet/HDB_DGT/2021-11-23 15:38:33/run_2021_11_23_15_38_33.log'
# file_

2021-11-23 16:27:42,896 INFO epoch: 22/484it: 130/371, lr: 0.009632, loss: 3.0833, boundery_bce_loss: 0.1284, boundery_dice_loss: 0.6320, eta: 45 days, 23:37:51, time: 43.7429
2021-11-23 16:28:26,006 INFO epoch: 22/484it: 140/371, lr: 0.009631, loss: 3.0869, boundery_bce_loss: 0.1235, boundery_dice_loss: 0.6475, eta: 43 days, 7:38:01, time: 43.1102
2021-11-23 16:29:08,198 INFO epoch: 22/484it: 150/371, lr: 0.009631, loss: 3.0488, boundery_bce_loss: 0.1065, boundery_dice_loss: 0.6178, eta: 40 days, 23:55:04, time: 42.1920
2021-11-23 16:29:49,525 INFO epoch: 22/484it: 160/371, lr: 0.009630, loss: 2.7134, boundery_bce_loss: 0.1113, boundery_dice_loss: 0.6117, eta: 38 days, 22:56:16, time: 41.3272
2021-11-23 16:30:31,823 INFO epoch: 22/484it: 170/371, lr: 0.009630, loss: 3.0231, boundery_bce_loss: 0.1112, boundery_dice_loss: 0.6096, eta: 37 days, 4:02:21, time: 42.2975
2021-11-23 16:31:13,475 INFO epoch: 22/484it: 180/371, lr: 0.009629, loss: 3.4781, boundery_bce_loss: 0.1214, boundery_dic

In [ ]:
file_ = list(filter(lambda x: x.endswith('.log'), os.listdir(logdir)))[0]
file_ = os.path.join(logdir, file_)
if not flag and osp.isfile(file_):
    with open(file_, "r") as f:
        str_ = f.readlines()[-24]
        st__, end__ = str_.find('Epoch') + 6, str_.find(' Iter')
        if st__ > -1 and end__ > -1 and local_max_epoch - 1 == int(str_[st__:end__]):
            start_epoch = local_max_epoch
            local_max_epoch += 2

start_epoch, local_max_epoch, best_iou

(41, 43, 0.4278781883209911)

## Training

In [ ]:
st = glob_st = time.time()
flag = False
for epoch_id in range(start_epoch, local_max_epoch):
    for images, labels in dl:
        it += 1
        start_ts = time.time()
        
        model.train()

        images = images.to(device)
        labels = labels.to(device)
        labels = torch.squeeze(labels, 1)
 
        optim.zero_grad()

        out_ffm, out_l3, out_l4 = model(images)
 
        loss_ffm = criteria_ffm(out_ffm, labels)
        loss_l4  = criteria_layer4(out_l4, labels)
        boundery_bce, boundery_dice = boundary_loss_func(out_l3, labels)

        boundery_bce_loss = boundery_bce
        boundery_dice_loss = boundery_dice

        loss = loss_ffm + loss_l4 + boundery_bce_loss + boundery_dice_loss

        loss.backward()
        optim.step()

        loss_avg.append(loss.item())

        loss_boundery_bce.append(boundery_bce_loss.item())
        loss_boundery_dice.append(boundery_dice_loss.item())

        if (it + 1) % print_interval == 0:
            loss_avg = sum(loss_avg) / len(loss_avg)
            lr = optim.lr
            ed = time.time()
            t_intv, glob_t_intv = ed - st, ed - glob_st
            eta = int((max_iter - it) * (glob_t_intv / it))
            eta = str(timedelta(seconds=eta))

            loss_boundery_bce_avg = sum(loss_boundery_bce) / len(loss_boundery_bce)
            loss_boundery_dice_avg = sum(loss_boundery_dice) / len(loss_boundery_dice)
            msg = ', '.join([
                'epoch: {epoch}/{max_epoch}'
                'it: {it}/{max_it}',
                'lr: {lr:4f}',
                'loss: {loss:.4f}',
                'boundery_bce_loss: {boundery_bce_loss:.4f}',
                'boundery_dice_loss: {boundery_dice_loss:.4f}',
                'eta: {eta}',
                'time: {time:.4f}',
            ]).format(
                epoch = epoch_id,
                max_epoch = max_epoch,
                it = it+1,
                max_it = epoch_iteration,
                lr = lr,
                loss = loss_avg,
                boundery_bce_loss = loss_boundery_bce_avg,
                boundery_dice_loss = loss_boundery_dice_avg,
                time = t_intv,
                eta = eta
            )
            
            logger.info(msg)
            print("loss/train_loss", loss.item(), it + 1)
            loss_avg = []
            loss_boundery_bce = []
            loss_boundery_dice = []
            st = ed

        if ((it + 1) % val_interval == 0 and it + 10 < epoch_iteration) or (it + 1) % epoch_iteration == 0:
            print('validation')
            torch.cuda.empty_cache()
            model.eval()
            loss_all = 0
            loss_n = 0
            with torch.no_grad():
                for i_val, (images_val, labels_val) in enumerate(dlval):
                    if (i_val + 1) % 50 == 0:
                        print(i_val + 1)

                    images_val = images_val.to(device)
                    labels_val = labels_val.to(device)
                    labels_val = torch.squeeze(labels_val, 1)

                    outputs = model(images_val)[0]
                    val_loss = criteria_val(outputs, labels_val)

                    pred = outputs.data.max(1)[1].cpu().numpy()
                    gt = labels_val.data.cpu().numpy()

                    running_metrics_val.update(gt, pred)
                    val_loss_meter.update(val_loss.item())

            print("loss/val_loss", val_loss_meter.avg)
            logger.info("Epoch %3d Iter %d Val Loss: %.4f" % (epoch_id, it + 1, val_loss_meter.avg))

            score, class_iou = running_metrics_val.get_scores()
            for k, v in score.items():
                print(k, v)
                logger.info("{}: {}".format(k, v))
                print("val_metrics/{}".format(k), v)

            for k, v in class_iou.items():
                logger.info("{}: {}".format(k, v))
                print("val_metrics/cls_{}".format(k), v)

            val_loss_meter.reset()
            running_metrics_val.reset()

            state = {
                    "epoch": epoch_id,
                    "iteration": it+ 1,
                    "model_state": model.state_dict(),
                    "optimizer_state": optim.get_state(),
            }
            save_path = os.path.join(
                writer.file_writer.get_logdir(),
                "{}_{}_checkpoint.pkl".format(model_arch, cfg_data['dataset']),
            )
            torch.save(state, save_path)

            if score["Mean IoU : \t"] >= best_iou:
                best_iou = score["Mean IoU : \t"]
                state = {
                    "epoch": epoch_id,
                    "iteration":it+ 1,
                    "model_state": model.state_dict(),
                    "best_iou": best_iou,
                }
                save_path = os.path.join(
                    writer.file_writer.get_logdir(),
                    "{}_{}_best_model.pkl".format(model_arch, cfg_data['dataset']),
                )
                torch.save(state, save_path)
            torch.cuda.empty_cache()
    it = 0

loss/train_loss 3.4885222911834717 10
loss/train_loss 3.0776875019073486 20
loss/train_loss 3.1200971603393555 30
loss/train_loss 2.9808926582336426 40
loss/train_loss 2.7536351680755615 50
loss/train_loss 3.56929874420166 60
loss/train_loss 2.966944694519043 70
loss/train_loss 2.6964364051818848 80
loss/train_loss 3.0642142295837402 90
loss/train_loss 2.96091365814209 100
loss/train_loss 2.698448419570923 110
loss/train_loss 2.9087462425231934 120
loss/train_loss 3.096327066421509 130
loss/train_loss 2.661802291870117 140
loss/train_loss 3.1281838417053223 150
loss/train_loss 2.584542751312256 160
loss/train_loss 3.3675496578216553 170
loss/train_loss 3.103978157043457 180
loss/train_loss 3.207629680633545 190
loss/train_loss 2.679269313812256 200
loss/train_loss 2.7626495361328125 210
loss/train_loss 2.8811488151550293 220
loss/train_loss 2.3620355129241943 230
loss/train_loss 2.3911843299865723 240
loss/train_loss 2.940701484680176 250
loss/train_loss 2.6687369346618652 260
loss/tra

In [ ]:
# with torch.no_grad():
#     for (images_val, labels_val, _) in valloader:
#         images_val = images_val.to(device)
#         labels_val = labels_val.to(device)

#         outputs = model(images_val)
#         outputs = output_val_upsample(outputs)
#         val_loss = loss_fn(input=outputs, target=labels_val)

#         pred = outputs.data.max(1)[1].cpu().numpy()
#         gt = labels_val.data.cpu().numpy()

#         running_metrics_val.update(gt, pred)
#         val_loss_meter.update(val_loss.item())

# writer.add_scalar("loss/val_loss", val_loss_meter.avg, i + 1)
# logger.info("Iter %d Val Loss: %.4f" % (i + 1, val_loss_meter.avg))

# score, class_iou = running_metrics_val.get_scores()